In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium_stealth import stealth

import time
import random
import string
import json
import pafy

import pandas as pd
import numpy as np

# Hide video unavailable warnings
import warnings
warnings.filterwarnings("ignore")

# Data Inladen

In [3]:
mails = pd.read_csv("../data/mails.csv")

In [4]:
videos = pd.read_csv("../data/uploads.csv")

# Helpers

In [5]:
def vid_to_watch(videos, usertype = 1):
    """Finds a conspiracy video to watch and 
    makes sure it's still accessible and under 2 hours long"""
    
    if usertype == 1:
        # Choose a non-conspiracy video to watch
        to_watch = np.random.choice(videos[~videos["conspiracy"]]["video_id"])
    else:
        # Choose a conspiracy video to watch
        to_watch = np.random.choice(videos[videos["conspiracy"]]["video_id"])
    
    # Check its length
    url = f"http://www.youtube.com/watch?v={to_watch}"
    try:
        video = pafy.new(url);
        vid_len = video.length
    except: # Video no longer accessible
        vid_len = 9999
    
    # Videos over 2 hours will be skipped
    while vid_len > 7200:
        to_watch = np.random.choice(videos[videos["conspiracy"]]["video_id"])
        
        url = f"http://www.youtube.com/watch?v={to_watch}"
        try:
            video = pafy.new(url);
            vid_len = video.length
        except:
            vid_len = 9999
            
    # Calculate how much of the video will be watched
    watch_time = np.random.normal(0.6, 0.15) * vid_len

    return watch_time, to_watch

In [6]:
def init_driver():
    """Initializes a selenium driver in stealth mode"""
        
    # Add options
    options = webdriver.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-blink-features")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-infobars")

    options.add_argument("start-maximized")

    # options.add_argument("--headless")

    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    # Start driver
    driver = webdriver.Chrome(options=options)

    # Hide the fact we're using a bot
    stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
            )
    
    return driver

In [20]:
def login_google(driver, mail, password):
    """Logs in on Google given a driver, email-address and password.
    This code only works if the email-address was made within chromedriver itself"""
    
    # login on google
    driver.get("https://accounts.google.com/ServiceLogin")
    time.sleep(np.random.uniform(1, 1.5))

    # Fillin mail
    driver.find_element_by_id("identifierId").send_keys(mail)
    driver.find_element_by_id("identifierNext").click()

    time.sleep(np.random.uniform(1, 1.5))

    # Fill in password
    driver.find_element_by_xpath("//input[@name = 'password']").send_keys(password)
    driver.find_element_by_xpath('//button[contains(@class, "VfPpkd-LgbsSe")]').click()

    time.sleep(0.5)

    # If security-check is asked, skip it
    try:
        driver.find_element_by_xpath("//div[contains(@class, 'U26fgb O0WRkf')]").click()
    except:
        pass

In [8]:
def in_seconds(timestamp):
    """Converts a YouTube timestamp to a number of seconds"""
    # Order times in ascending order
    times = timestamp.split(":")[::-1]
    
    # Convert to seconds
    return sum([int(times[i]) * 60**i for i in range(len(times))])

In [19]:
def prepare_video(driver):
    """Does everything necessary to start watching a video"""

    # Skip ad(s) if they are there
    for ad in range(2):
        time.sleep(0.5)
        if driver.find_elements_by_xpath("//img[@class = 'ytp-ad-image']"):
            time.sleep(5.5)
            try:
                driver.find_element_by_xpath("//div[@class = 'ytp-ad-text ytp-ad-skip-button-text']").click()
            except:
                pass

    # Toggle autoplayer
    time.sleep(1)
    autoplay = driver.find_element_by_xpath("//div[@class = 'ytp-autonav-toggle-button']")
    
    if autoplay.get_attribute("aria-checked") == "true":
        autoplay.click()

In [10]:
def check_video_running(driver, watch_time):
    timestamp = driver.find_element_by_class_name("ytp-time-current").text
            
    # Sometimes the timestamp cannot be displayed
    if timestamp:   
        return in_seconds(timestamp) < watch_time
                    
    # Check if the video is done (can happen when ads play and the video is very short)
    try:
        driver.find_element_by_xpath("//div[contains(@class, 'ended-mode')]")
        return False
    except:
        pass
    
    # Video is still running
    return True

In [11]:
def choose_recommendation(recommendations, usertype = 3):
    """Selects a video to watch based on a top 10 of given recommendations"""
    
    # These odds are not based on anything yet.
    # TODO: Find a paper that has done propensity weighting on YouTube (or a similar site)
    choice = np.random.choice(recommendations, p = [0.4, 0.2, 0.1, 0.1, 0.05, 0.05, 0.05, 0.025, 0.0125, 0.0125])
    
    vid_len = pafy.new(choice).length
            
    # Calculate how much of the video will be watched
    watch_time = np.random.normal(0.6, 0.15) * vid_len
    
    return watch_time, choice.replace("https://www.youtube.com/watch?v=", "")

# Het experiment

In [17]:
def run_experiment(videos, mails, usertype=1):
    experiment_results = {"user":[], "vids_watched":[], "video":[], "channel":[]}

    for user in mails.itertuples():
        # Find current user's info
        mail, password = user[1], user[2]

        # Initialize driver
        driver = init_driver()    

        # Login to google
        login_google(driver, mail, password)
        
        rec_to_watch = None

        # Start watching videos
        for i in range(1, 3):
            if not rec_to_watch:
                # Get a random video from the dataset and watch it
                watch_time, to_watch = vid_to_watch(videos, usertype)
                driver.get(f"http://youtube.com/watch?v={to_watch}")
                vid_running = True
            else:
                driver.get(f"http://youtube.com/watch?v={rec_to_watch}")
                vid_running = True

            # Skip ads, disable autoplay
            prepare_video(driver)

            # Watch video
            while check_video_running(driver, watch_time):
                time.sleep(1)
            
            # If we have a usertype that relies on direct recommendations
            if usertype == 3:
                video_recs = driver.find_elements_by_xpath("//a[contains(@class, 'ytd-compact-video-renderer')]")
                watch_time, rec_to_watch = choose_recommendation([rec.get_attribute("href")
                                                                  for rec in video_recs][:10],
                                                                 usertype)

            # Go to the youtube homepage
            driver.get("http://youtube.com")
            time.sleep(1)

            # Get videos on youtube home
            channels = driver.find_elements_by_xpath("//a[@id = 'avatar-link']")
            vids = driver.find_elements_by_xpath("//a[@id = 'video-title-link']")

            # If we have a user that relies on homepage recommendations
            if usertype == 4:
                watch_time, rec_to_watch = choose_recommendation([rec.get_attribute("href")
                                                                  for rec in vids][:10],
                                                                 usertype)
            
            # Get top 15 recommendations
            for rec in range(15):
                experiment_results["user"].append(mail)
                experiment_results["vids_watched"].append(i)

                experiment_results["video"].append(vids[rec].get_attribute("href"))
                experiment_results["channel"].append(channels[rec].get_attribute("href"))
        break

    return pd.DataFrame(experiment_results)

run_experiment(videos, mails, 3)

ERROR: Video unavailable
This video has been removed by the uploader


,user,vids_watched,video,channel
0,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=gygGZkMykRE,https://www.youtube.com/user/InTouchMinistries
1,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=P3Mfkv3RJjY,https://www.youtube.com/channel/UC3Sax3ARJDZut...
2,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=543Eu0qLbT0,https://www.youtube.com/c/JosephPrinceOnline
3,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=8vQjI3FqbQ0,https://www.youtube.com/channel/UCuEGJ2eBN6wz8...
4,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=yaWVflQolmM,https://www.youtube.com/c/Richroll
5,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=xF_VYNtDgN8,https://www.youtube.com/user/theofficial700club
6,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=ABzoi5aPeeY,https://www.youtube.com/c/ChristChurchLondon0
7,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=MZa_bdYOWu8&li...,None
8,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=S4gj0CsCr54,https://www.youtube.com/c/991JOYFM
9,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=6B_6K-splRU,https://www.youtube.com/c/CosmoSapiens
